In [1]:
import os
from langchain import OpenAI as LangChainOpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv
from langchain.prompts import ChatPromptTemplate

In [2]:
# Load env variables
load_dotenv(find_dotenv())

True

In [3]:
# Initialize the OpenAI model instance using LangChain
llm = LangChainOpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=os.environ["OPENAI_API_KEY"])

c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x0000023B4047F650>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000023B404B5430>, openai_api_key='sk-Hjsnww-7kJuSkfrRZb7aMY1ADeZh5fP6B0bvnd-0WwT3BlbkFJ7cx1ZsmSgUsimu7lA0v05xL05hOe0TgUQQMfzJrA4A', openai_proxy='')

In [5]:
# Use the  model to generate a joke
joke = llm("Tell me a joke")
joke

c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [6]:
# Define the template for evaluating sentiment and subject
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

In [7]:
# Create a prompt template using LangChain's ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template=template)

In [34]:
# Create an LLMChain with the prompt and the model
chain = LLMChain(llm=llm, prompt=prompt_template)

In [35]:
# Predict the sentiment and subject of the input text
output = chain.predict(input="I created a AI automated Chatbot today and it was okay")
output

'\nOutput: \n{\n    "sentiment": "neutral",\n    "subject": "technology"\n}'

In [36]:
# Alternative new version
# this creates a runnable sequence that first processes the input through the prompt_template
# and then passes the result to the llm
chain = prompt_template | llm
# invoke method uses this chain
output = chain.invoke({"input": "I created an AI automated Chatbot today and it was okay."})
output

'\nsentiment: neutral\nsubject: technology\n\n{\n    "sentiment": "neutral",\n    "subject": "technology"\n}'

### Real World example with ResponseSchema, Templates, Chains and OutputParsers
`There were two issues with the output: It contains text and the output is just a string instead of a dict`

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chat_models import ChatOpenAI


In [ ]:
# Define response schema (structured output) for parsing
# key and value pair for sentiment, subjectm and price schema
# description defined as // in the format_instructions
sentiment_schema = ResponseSchema(
    name="sentiment",
    description="Is the text positive, neutral or negative? Only provide these words."
)

subject_schema = ResponseSchema(
    name="subject",
    description="What subject is the text about? Use exactly one word."
)
price_schema = ResponseSchema(
    name="price",
    description="How expensive was the product? Use None if no price was provided in the text."
)
# list
response_schemas = [sentiment_schema, subject_schema, price_schema]

In [ ]:
# Initialize the structured output parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# Get format instructions for the parser
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Is the text positive, neutral or negative? Only provide these words.\n\t"subject": string  // What subject is the text about? Use exactly one word.\n\t"price": string  // How expensive was the product? Use None if no price was provided in the text.\n}\n```'

In [ ]:
# Define the template with format instructions included
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.
price: How expensive was the product? Use None if no price was provided in the text.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

{format_instructions}

text: {input}
"""

In [ ]:
# Create the prompt template
prompt = ChatPromptTemplate.from_template(template=template)

In [ ]:
# Escape the format_instructions string
# treat {} as literal text for e.g. {'sentiment': 'positive', 'subject': 'food', 'price': 'None'}
escaped_format_instructions = format_instructions.replace('{', '{{').replace('}', '}}')

# Format the messages with input and format instructions
messages = prompt.format_messages(
    input="I ordered Chicken and Coleslaw from Texas it was spicy.",
    format_instructions=escaped_format_instructions
)

In [ ]:
# Initialize the OpenAI chat model with zero temperature for deterministic output
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
response = chat(messages)
parsed_output = parser.parse(response.content)
parsed_output


c:\Users\shahi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'sentiment': 'positive', 'subject': 'food', 'price': 'None'}